## Who said Python was slow? 

Leveraging python built-in functions and dictionaries, that are in fact hash-tables, we can translate the titles of our shopee dataset in no time! <br> Probably benefiting our Bert-based models. Thank's to Allohvk and his great notebook <br>
https://www.kaggle.com/allohvk/offline-translator-indonesean-to-english-reverse

In [14]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [15]:
df = pd.read_csv("input/train_st_fold42.csv")
df.head(10)

,posting_id,image,image_phash,title,label_group,group_counts,psudo_label,fold
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,2,0,3.0
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,2,0,0.0
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,2,0,3.0
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,2,0,1.0
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,2,0,0.0
5,train_2464356923,0013e7355ffc5ff8fb1ccad3e42d92fe.jpg,bbd097a7870f4a50,CELANA WANITA (BB 45-84 KG)Harem wanita (bisa...,2660605217,3,1,4.0
6,train_1802986387,00144a49c56599d45354a1c28104c039.jpg,f815c9bb833ab4c8,Jubah anak size 1-12 thn,1835033137,7,2,3.0
7,train_1806152124,0014f61389cbaa687a58e38a97b6383d.jpg,eea7e1c0c04da33d,KULOT PLISKET SALUR /CANDY PLISKET /WISH KULOT...,1565741687,2,0,0.0
8,train_86570404,0019a3c6755a194cb2e2c12bfc63972e.jpg,ea9af4f483249972,"[LOGU] Tempelan kulkas magnet angka, tempelan ...",2359912463,3,1,3.0
9,train_831680791,001be52b2beec40ddc1d2d7fc7a68f08.jpg,e1ce953d1a70618f,BIG SALE SEPATU PANTOFEL KULIT KEREN KERJA KAN...,2630990665,2,0,3.0


# Create Dictionaries

In [16]:
def make_dictionaries():
    indo_to_eng_dict = {}
    eng_to_indo_dict = {}
    with open('indonesean_english_dict.txt') as f:
        lines = f.readlines()
        for line in tqdm(lines):
            if "\"" in line:
                print("ignoring: ", line) # skip non-sense line
                continue
            inputs = line[2:-3].split("', '")
            indo_word = inputs[0].lower()
            eng_word = inputs[1].lower()
            
            indo_to_eng_dict[indo_word] = eng_word
            eng_to_indo_dict[eng_word] = indo_word
    f.close()
    return indo_to_eng_dict, eng_to_indo_dict


In [17]:
INDO_TO_ENG_DICT, ENG_TO_INDO_DICT = make_dictionaries()

  0%|          | 0/2144 [00:00<?, ?it/s]

ignoring:  ('ALQURAN', "AL-QUR'AN")



In [18]:
i = 0
for key, val in INDO_TO_ENG_DICT.items():
    print(key, val)
    i+=1
    if i == 10:
        break

sikut elbow
petak map
jendela window
merek brand
gagas failed
timi thyme
gerigi teeth
landak porcupine
kasus case
itik duck


# Translate

In [19]:
def translate_indo_to_eng(text):
    words = text.lower().split()
    translated_words = list(map(lambda x : x if x not in INDO_TO_ENG_DICT else INDO_TO_ENG_DICT[x], words))
    return " ".join(translated_words)

In [20]:
def translate_eng_to_indo(text):
    words = text.lower().split()
    translated_words = list(map(lambda x : x if x not in ENG_TO_INDO_DICT else ENG_TO_INDO_DICT[x], words))
    return " ".join(translated_words)

In [21]:
df["title"]

0                                Paper Bag Victoria Secret
1        Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...
2              Maling TTS Canned Pork Luncheon Meat 397 gr
3        Daster Batik Lengan pendek - Motif Acak / Camp...
4                        Nescafe \xc3\x89clair Latte 220ml
                               ...                        
34245    Masker Bahan Kain Spunbond Non Woven 75 gsm 3 ...
34246      MamyPoko Pants Royal Soft - S 70 - Popok Celana
34247    KHANZAACC Robot RE101S 1.2mm Subwoofer Bass Me...
34248    Kaldu NON MSG HALAL Mama Kamu Ayam Kampung , S...
34249    FLEX TAPE PELAPIS BOCOR / ISOLASI AJAIB / ANTI...
Name: title, Length: 34250, dtype: object

In [23]:
df["title"] = df["title"].apply(translate_indo_to_eng)

In [11]:
df["title"] = df["title"].apply(translate_eng_to_indo)

In [25]:
df.to_csv("train_eng.csv",index=False)